In [1]:
import matlab.engine
import numpy as np
eng = matlab.engine.start_matlab()
import pandas as pd

In [2]:
def get_echo(img,company,R,C,save_file):
    import matplotlib.pyplot as plt
    import numpy as np
    import skimage.io as io
    import dicom
    from scipy import misc
    #img = plt.imread(img_add)
    
    
    dic_m = {}
    
    dic_m["'GEMS Ultrasound' 636 422"] = [[0,316],[311,0],[411,186],[421,306],[421,586],[304,629],[0,316]]
    dic_m["'GEMS Ultrasound' 636 434"] = [[10,320],[320,10],[402,162],[400,600],[124,578],[10,320]]
    dic_m["'GE Vingmed Ultrasound' 636 434"] = [[7,319],[322,13],[401,241],[401,582],[7,577]]
    dic_m["'Philips Medical Systems' 800 600"]=[[98,426],[560,6],[568,636],[432,758],[426,98]]
    
    dic_m["'GE Healthcare Ultrasound' 1016 708"] = [[66,510],[505,84],[650,313],[663,524],[574,985],[60,510]]
    dic_m["'SIEMENS' 1024 768"] = [[30,474],[469,0],[673,0],[673,1024],[469,1023],[30,551]]
    
    if(company.lower()=='gems ultrasound' and R == 636 and C == 422):
        coords = np.array([np.arange(6,316),np.arange(316,6,-1)]).T
        for i in coords:
            #print(i)
            img[:i[0],:i[1]]=0

        coords = np.array([np.arange(350,420),np.arange(316,6,-1)]).T
        
        img[387:,:170]=0
        img[390:,603:]=0
        img[8:173,602:]=0
        
    elif (company.lower()=='siemens' and R == 1024 and C == 768):
        coords = np.array([np.arange(469,30,-1),np.arange(30,469)]).T
        for i in coords:
            img[:i[0],:i[1]]=0
        img[674:,:997]=0
        coords1 = np.array([np.arange(36,469),np.arange(561,993)]).T
        img[:258,755:] = 0
    
    elif company.lower()=='ge healthcare ultrasound' and R == 1016 and C == 708 :
        img[:148,:259] = 0
        ekg = img[650:,:]
        img[650:,:] = 0
        img[:,915:] = 0
        
    elif company.lower()=='gems ultrasound' and R == 636 and C == 434:
        img[:8,:] = 0
        img[0:42,:95]=0
        img[:63,:131] = 0
        img[:10,:] = 0
        img[:207,592:] = 0
        ekg = img[395:,:]
        img[393:,:] = 0
        
    elif(company.lower()=='ge vingmed ultrasound' and R == 636  and C == 434):
        img[:8,:] = 0
        img[0:42,:95]=0
        img[:63,:131] = 0
        img[:10,:] = 0
        img[:207,592:] = 0
        ekg = img[395:,:]
        img[393:,:] = 0
        
    elif(company.lower()=='philips medical systems' and R==800 and C ==600):
        img[:130,0:900]=0
        #print('here')
        img[:,:94]=0
        img[61:191,:220] = 0        
        img[:93,:431] = 0
        img[:380,746:,] = 0
        img[538:,679:] = 0
        img[555:,:]=0
    
    #imga = plt.imshow(img, interpolation='nearest',aspect='auto',cmap='gray')
    #plt.axis('off')
    #plt.tight_layout()
    #plt.savefig('/data/Gurpreet/VC/imgtemp.jpg', bbox_inches='tight')
    #io.imsave(save_file,img)
    
    misc.imsave(save_file,img)
    
    return img

In [3]:
Testfiles=pd.read_excel('/data/Gurpreet/Echo/View_point2.xlsx', sheetname='Test', header=0)

import scipy.io
for i in xrange(0,1):
    patnumber=Testfiles.iloc[i,0]
    Videonumber=Testfiles.iloc[i,1]
    Viewplane=Testfiles.iloc[i,3]
    imgname="/data/Gurpreet/Echo/"+str(patnumber)+"/"+str(patnumber)+"("+str(Videonumber)+").dcm"
    [MANUFACTURER,WIDTH,HEIGHT]=eng.get_file_and_info3(imgname,220,nargout=3)
    tempfile="/home/gus2011/tempimage_220.mat"
    IMG=scipy.io.loadmat(tempfile, mat_dtype=True)['IMG']
    #!rm /home/gus2011/tempimage_210.mat
    #IMG=np.asarray(IMG)
    SoI=IMG.shape
    Totalframes=SoI[3]
    print imgname+"    "+Viewplane
    for j in xrange(1,Totalframes):
        IMG2=IMG[:,:,:,j]
        savename="/data/Gurpreet/Echo/RMOCR/TEST/"+str(Viewplane)+"/"+str(patnumber)+"_"+str(Videonumber)+"_"+str(j)+".jpg"
        IM=get_echo(IMG2,MANUFACTURER,WIDTH,HEIGHT,savename)

/data/Gurpreet/Echo/2/2(1).dcm    PLA


In [4]:
Trainfiles=pd.read_excel('/data/Gurpreet/Echo/View_point2.xlsx', sheetname='Train', header=0)
import scipy.io
for i in xrange(1,((Trainfiles.size/4))):
    patnumber=Trainfiles.iloc[i,0]
    Videonumber=Trainfiles.iloc[i,1]
    Viewplane=Trainfiles.iloc[i,3]
    imgname="/data/Gurpreet/Echo/"+str(patnumber)+"/"+str(patnumber)+"("+str(Videonumber)+").dcm"
    [MANUFACTURER,WIDTH,HEIGHT]=eng.get_file_and_info3(imgname,210,nargout=3)
    tempfile="/home/gus2011/tempimage_210.mat"
    IMG=scipy.io.loadmat(tempfile, mat_dtype=True)['IMG']
    #!rm /home/gus2011/tempimage_210.mat
    #IMG=np.asarray(IMG)
    SoI=IMG.shape
    Totalframes=SoI[3]
    print imgname+"    "+Viewplane
    for j in xrange(0,Totalframes):
        IMG2=IMG[:,:,:,j]
        savename="/data/Gurpreet/Echo/RMOCR/TRAIN/"+str(Viewplane)+"/"+str(patnumber)+"_"+str(Videonumber)+"_"+str(j)+".jpg"
        IM=get_echo(IMG2,MANUFACTURER,WIDTH,HEIGHT,savename)

/data/Gurpreet/Echo/1/1(8).dcm    PLA
/data/Gurpreet/Echo/1/1(11).dcm    PSA
/data/Gurpreet/Echo/1/1(16).dcm    PSA
/data/Gurpreet/Echo/1/1(17).dcm    PSA
/data/Gurpreet/Echo/1/1(18).dcm    PSA
/data/Gurpreet/Echo/1/1(20).dcm    PSA
/data/Gurpreet/Echo/1/1(21).dcm    PSA
/data/Gurpreet/Echo/1/1(22).dcm    PSA
/data/Gurpreet/Echo/1/1(23).dcm    PSA
/data/Gurpreet/Echo/1/1(24).dcm    A4C
/data/Gurpreet/Echo/1/1(30).dcm    A3C
/data/Gurpreet/Echo/1/1(38).dcm    A4C
/data/Gurpreet/Echo/1/1(39).dcm    A3C
/data/Gurpreet/Echo/1/1(40).dcm    A3C
/data/Gurpreet/Echo/1/1(41).dcm    A2C
/data/Gurpreet/Echo/1/1(42).dcm    A2C
/data/Gurpreet/Echo/1/1(48).dcm    SCX
/data/Gurpreet/Echo/1/1(49).dcm    SCX
/data/Gurpreet/Echo/1/1(51).dcm    SSX
/data/Gurpreet/Echo/17/17(1).dcm    PLA
/data/Gurpreet/Echo/17/17(2).dcm    PLA
/data/Gurpreet/Echo/17/17(6).dcm    PSA
/data/Gurpreet/Echo/17/17(13).dcm    PSA
/data/Gurpreet/Echo/17/17(14).dcm    PSA
/data/Gurpreet/Echo/17/17(15).dcm    PSA
/data/Gurpreet/Ec

/data/Gurpreet/Echo/46/46(43).dcm    SCX
/data/Gurpreet/Echo/46/46(44).dcm    SCX
/data/Gurpreet/Echo/54/54(1).dcm    PLA
/data/Gurpreet/Echo/54/54(5).dcm    PLA
/data/Gurpreet/Echo/54/54(8).dcm    PSA
/data/Gurpreet/Echo/54/54(13).dcm    PSA
/data/Gurpreet/Echo/54/54(14).dcm    PSA
/data/Gurpreet/Echo/54/54(15).dcm    PSA
/data/Gurpreet/Echo/54/54(17).dcm    PSA
/data/Gurpreet/Echo/54/54(18).dcm    PSA
/data/Gurpreet/Echo/54/54(19).dcm    PSA
/data/Gurpreet/Echo/54/54(20).dcm    A4C
/data/Gurpreet/Echo/54/54(24).dcm    A3C
/data/Gurpreet/Echo/54/54(29).dcm    A4C
/data/Gurpreet/Echo/54/54(30).dcm    A3C
/data/Gurpreet/Echo/54/54(31).dcm    A2C
/data/Gurpreet/Echo/54/54(32).dcm    A2C
/data/Gurpreet/Echo/54/54(36).dcm    A4C
/data/Gurpreet/Echo/57/57(1).dcm    PLA
/data/Gurpreet/Echo/57/57(2).dcm    PLA
/data/Gurpreet/Echo/57/57(6).dcm    PLA
/data/Gurpreet/Echo/57/57(8).dcm    PSA
/data/Gurpreet/Echo/57/57(15).dcm    PSA
/data/Gurpreet/Echo/57/57(16).dcm    PSA
/data/Gurpreet/Echo/57/

/data/Gurpreet/Echo/80/80(35).dcm    A2C
/data/Gurpreet/Echo/80/80(40).dcm    A4C
/data/Gurpreet/Echo/80/80(47).dcm    SCX
/data/Gurpreet/Echo/80/80(49).dcm    SCX
/data/Gurpreet/Echo/80/80(50).dcm    SSX
/data/Gurpreet/Echo/82/82(1).dcm    PLA
/data/Gurpreet/Echo/82/82(5).dcm    PSA
/data/Gurpreet/Echo/82/82(9).dcm    PSA
/data/Gurpreet/Echo/82/82(10).dcm    PSA
/data/Gurpreet/Echo/82/82(11).dcm    PSA
/data/Gurpreet/Echo/82/82(12).dcm    PSA
/data/Gurpreet/Echo/82/82(14).dcm    PSA
/data/Gurpreet/Echo/82/82(15).dcm    PSA
/data/Gurpreet/Echo/82/82(16).dcm    PSA
/data/Gurpreet/Echo/82/82(17).dcm    PSA
/data/Gurpreet/Echo/82/82(18).dcm    A4C
/data/Gurpreet/Echo/82/82(23).dcm    A3C
/data/Gurpreet/Echo/82/82(26).dcm    A2C
/data/Gurpreet/Echo/82/82(30).dcm    A4C
/data/Gurpreet/Echo/82/82(31).dcm    A3C
/data/Gurpreet/Echo/82/82(32).dcm    A2C
/data/Gurpreet/Echo/85/85(1).dcm    PLA
/data/Gurpreet/Echo/85/85(9).dcm    PSA
/data/Gurpreet/Echo/85/85(10).dcm    PSA
/data/Gurpreet/Echo/8

/data/Gurpreet/Echo/102/102(31).dcm    A3C
/data/Gurpreet/Echo/102/102(35).dcm    A2C
/data/Gurpreet/Echo/102/102(38).dcm    A4C
/data/Gurpreet/Echo/102/102(39).dcm    A3C
/data/Gurpreet/Echo/102/102(40).dcm    A2C
/data/Gurpreet/Echo/102/102(41).dcm    A4C
/data/Gurpreet/Echo/102/102(46).dcm    A4C
/data/Gurpreet/Echo/102/102(50).dcm    SCX
/data/Gurpreet/Echo/102/102(51).dcm    SCX
